## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.
D:\Anaconda3\envs\envirment_1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\envs\envirment_1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\envs\envirment_1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\envs\envirment_1\lib\site-packages\tensorflow\python

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization, Dense , Input
from keras.models import Model
def build_mlp(input_shape, output_shape = 10, num_nur = [512,256,128]):
    """Code Here
    建立你的神經網路
    """
    inp = Input(input_shape)
    for i , nu_num in enumerate(num_nur):
        if i == 0:
            x = Dense(units = nu_num , activation = 'relu', name = 'Hidden_Layer'+str(i+1))(inp)
            x = BatchNormalization()(x)
        else:
            x = Dense(units = nu_num , activation = 'relu', name = 'Hidden_Layer'+str(i+1))(x)
            x = BatchNormalization()(x)
    output = Dense(units = output_shape , activation = 'softmax' , name = 'Output_Layer')(x)
    
    model = Model(inputs = [inp], outputs = [output])
    return model

In [8]:
"""Code Here
設定超參數
"""
batch_size = 256
epochs = 50
learning_rate = 0.001
momentum = 0.95
save_best_only = [True, False]

In [14]:
# 載入 Callbacks
from keras.callbacks import ModelCheckpoint
chpoint = ModelCheckpoint(filepath='./model1.h5', monitor = 'val_loss', save_best_only = True)
"""Code Here
設定 callbacks: model checkpoint
"""
results = {}
for i in save_best_only:
    keras.backend.clear_session()
    model = build_mlp(x_train.shape[1:])
    model.summary()
    opt = keras.optimizers.SGD(lr = learning_rate, momentum = momentum , nesterov =True)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics=['accuracy'])
    chpoint = ModelCheckpoint(filepath='./model_'+str(i)+'.h5', monitor = 'val_loss', save_best_only = i)
    model.fit(x_train, y_train , batch_size = batch_size , epochs = epochs , shuffle = True , validation_data = (x_test , y_test), callbacks=[chpoint])
    
    #train_loss = model.history.history['loss']
    #val_loss = model.history.history['val_loss']
    #train_acc = model.history.history['accuracy']
    #val_acc = model.history.history['val_accuracy']
    #results['experi_save_best:'+str(i)] = {'train_loss': train_loss , 'val_loss' : val_loss , 'train_acc': train_acc, 'val_acc': val_acc}

    model = keras.models.load_model('./model_'+str(i)+'.h5')
    loss_loadback, acc_loadback = model.evaluate(x_test, y_test)
    results['experi_save_best:'+str(i)] = {'loss_loadback': loss_loadback, 'acc_loadback': acc_loadback}
    
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
Hidden_Layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
Hidden_Layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
Hidden_Layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512 

Epoch 46/50
50000/50000 [==============================] - 14s 286us/step - loss: 0.2205 - accuracy: 0.9353 - val_loss: 2.3603 - val_accuracy: 0.4678
Epoch 47/50
50000/50000 [==============================] - 13s 267us/step - loss: 0.2039 - accuracy: 0.9424 - val_loss: 2.4218 - val_accuracy: 0.4662
Epoch 48/50
50000/50000 [==============================] - 15s 299us/step - loss: 0.1901 - accuracy: 0.9470 - val_loss: 2.3816 - val_accuracy: 0.4725
Epoch 49/50
50000/50000 [==============================] - 15s 303us/step - loss: 0.1849 - accuracy: 0.9494 - val_loss: 2.4458 - val_accuracy: 0.4721
Epoch 50/50
10000/10000 [==============================] - 1s 116us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
Hidden_Layer1 (Dense)        (None, 512)      

50000/50000 [==============================] - 14s 286us/step - loss: 0.2941 - accuracy: 0.9100 - val_loss: 2.2102 - val_accuracy: 0.4666
Epoch 41/50
50000/50000 [==============================] - 15s 296us/step - loss: 0.2871 - accuracy: 0.9125 - val_loss: 2.1498 - val_accuracy: 0.4758
Epoch 42/50
50000/50000 [==============================] - 15s 305us/step - loss: 0.2631 - accuracy: 0.9227 - val_loss: 2.1628 - val_accuracy: 0.4780
Epoch 43/50
50000/50000 [==============================] - 17s 330us/step - loss: 0.2503 - accuracy: 0.9264 - val_loss: 2.3235 - val_accuracy: 0.4655
Epoch 44/50
50000/50000 [==============================] - 15s 295us/step - loss: 0.2425 - accuracy: 0.9303 - val_loss: 2.3295 - val_accuracy: 0.4586
Epoch 45/50
50000/50000 [==============================] - 15s 305us/step - loss: 0.2215 - accuracy: 0.9370 - val_loss: 2.3649 - val_accuracy: 0.4761
Epoch 46/50
50000/50000 [==============================] - 16s 323us/step - loss: 0.2120 - accuracy: 0.9401 - va

'Code Here\n撰寫你的訓練流程並將結果用 dictionary 紀錄\n'

In [15]:
print('best:',results['experi_save_best:'+'True'])
print('no_best:',results['experi_save_best:'+'False'])

best: {'loss_loadback': 1.4567254888534547, 'acc_loadback': 0.4975000023841858}
no_best: {'loss_loadback': 2.535802745056152, 'acc_loadback': 0.4611000120639801}


In [25]:
### Homework_2 ###


batch_size = 256
epochs = 50
learning_rate = 0.001
momentum = 0.95

### train and save best model weights ###
model = build_mlp(x_train.shape[1:])
model.summary()
opt = keras.optimizers.SGD(lr = learning_rate, momentum = momentum , nesterov =True)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics=['accuracy'])
chpoint = ModelCheckpoint(filepath='./model2.h5', monitor = 'val_loss', save_best_only = True, save_weights_only = True)
model.fit(x_train, y_train , batch_size = batch_size , epochs = epochs , shuffle = True , validation_data = (x_test , y_test), callbacks=[chpoint])


##### read model weights #####
model_new = build_mlp(x_train.shape[1:])
model_new.load_weights('./model2.h5')


##### predict #####
model_new.predict(x_test)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
Hidden_Layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_22 (Batc (None, 512)               2048      
_________________________________________________________________
Hidden_Layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_23 (Batc (None, 256)               1024      
_________________________________________________________________
Hidden_Layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_24 (Batc (None, 128)               512 

Epoch 46/50
50000/50000 [==============================] - 13s 262us/step - loss: 0.2142 - accuracy: 0.9392 - val_loss: 2.3712 - val_accuracy: 0.4717
Epoch 47/50
50000/50000 [==============================] - 13s 265us/step - loss: 0.2032 - accuracy: 0.9434 - val_loss: 2.4698 - val_accuracy: 0.4627
Epoch 48/50
50000/50000 [==============================] - 14s 283us/step - loss: 0.1977 - accuracy: 0.9439 - val_loss: 2.3391 - val_accuracy: 0.4685
Epoch 49/50
50000/50000 [==============================] - 14s 275us/step - loss: 0.1760 - accuracy: 0.9529 - val_loss: 2.4804 - val_accuracy: 0.4594
Epoch 50/50
50000/50000 [==============================] - 15s 290us/step - loss: 0.1821 - accuracy: 0.9497 - val_loss: 2.5253 - val_accuracy: 0.4650


array([[6.9112680e-03, 1.7047573e-02, 1.4140731e-02, ..., 1.2706222e-03,
        1.1458580e-02, 6.8045635e-04],
       [6.1401103e-02, 1.9959563e-01, 3.9565545e-03, ..., 1.4701276e-02,
        3.4026638e-01, 3.6276236e-01],
       [2.8752813e-02, 4.6367034e-02, 1.6397068e-02, ..., 2.1697304e-03,
        8.2798195e-01, 7.1441181e-02],
       ...,
       [5.0205929e-04, 1.1084243e-02, 2.1260391e-01, ..., 2.2402735e-02,
        8.3116873e-04, 6.7863315e-03],
       [4.3329671e-03, 4.6298139e-02, 7.4965023e-02, ..., 1.8545244e-02,
        1.6554974e-03, 1.8454975e-02],
       [1.6229236e-02, 2.1600889e-03, 2.4043908e-02, ..., 8.2955033e-02,
        3.1895025e-03, 2.1235617e-03]], dtype=float32)